# Exam 2021, 8.00-13.00 for the course 1MS041 (Introduction to Data Science / Introduktion till dataanalys)

## Instructions:
1. Complete the problems by following instructions.
2. When done, submit this file with your solutions saved, following the instruction sheet.

This exam has 3 problems for a total of 40 points, to pass you need
20 points.

## Some general hints and information:
* Try to answer all questions even if you are uncertain.
* Comment your code, so that if you get the wrong answer I can understand how you thought
this can give you some points even though the code does not run.
* Follow the instruction sheet rigorously.
* This exam is partially autograded, but your code and your free text answers are manually graded anonymously.
* If there are any questions, please ask the exam guards, they will escalate it to me if necessary.
* I (Benny) will visit the exam room at around 10:30 to see if there are any questions.

## Tips for free text answers
* Be VERY clear with your reasoning, there should be zero ambiguity in what you are referring to.
* If you want to include math, you can write LaTeX in the Markdown cells, for instance `$f(x)=x^2$` will be rendered as $f(x)=x^2$ and `$$f(x) = x^2$$` will become an equation line, as follows
$$f(x) = x^2$$
Another example is `$$f_{Y \mid X}(y,x) = P(Y = y \mid X = x) = \exp(\alpha \cdot x + \beta)$$` which renders as
$$f_{Y \mid X}(y,x) = P(Y = y \mid X = x) = \exp(\alpha \cdot x + \beta)$$

## Finally some rules:
* You may not communicate with others during the exam, for example:
    * You cannot ask for help in Stack-Overflow or other such help forums during the Exam.
    * You may not communicate with AI's, for instance ChatGPT.
    * Your on-line and off-line activity is being monitored according to the examination rules.

## Good luck!

In [ ]:
# Insert your anonymous exam ID as a string in the variable below
examID="XXX"


---
## Exam vB, PROBLEM 1
Maximum Points = 8


## Probability warmup
Let's say we have an exam question which consists of $20$ yes/no questions. 
From past performance of similar students, a randomly chosen student will know the correct answer to $N \sim \text{binom}(20,11/20)$ questions. Furthermore, we assume that the student will guess the answer with equal probability to each question they don't know the answer to, i.e. given $N$ we define $Z \sim \text{binom}(20-N,1/2)$ as the number of correctly guessed answers. Define $Y = N + Z$, i.e., $Y$ represents the number of total correct answers.

We are interested in setting a deterministic threshold $T$, i.e., we would pass a student at threshold $T$ if $Y \geq T$. Here $T \in \{0,1,2,\ldots,20\}$.

1. [5p] For each threshold $T$, compute the probability that the student *knows* less than $10$ correct answers given that the student passed, i.e., $N < 10$. Put the answer in `problem11_probabilities` as a list.
2. [3p] What is the smallest value of $T$ such that if $Y \geq T$ then we are 90\% certain that $N \geq 10$?

In [5]:

# Hint the PMF of N is p_N(k) where p_N is
from scipy.special import binom as binomial
p = 11/20
p_N = lambda k: binomial(20,k)*((1-p)**(20-k))*(p**k)

In [6]:

# Part 1: 
# replace XXX to represent P(N < 10) for T = [0,1,2,...,20], i.e. your answer should be a list
# of length 21.

import numpy as np

# PMF of Z | N=n ~ Binom(20-n, 1/2)
def p_Z_given_N(z, n):
    return binomial(20 - n, z) * (0.5 ** (20 - n))

# Compute P(N < 10 | Y >= T) for all T
problem11_probabilities = []

for T in range(21):
    numerator = 0.0
    denominator = 0.0
    
    for n in range(21):
        for z in range(21 - n):
            y = n + z
            prob = p_N(n) * p_Z_given_N(z, n)
            
            if y >= T:
                denominator += prob
                if n < 10:
                    numerator += prob

    problem11_probabilities.append(numerator / denominator)

problem11_probabilities

[np.float64(0.24928935982841186),
 np.float64(0.24928935982832884),
 np.float64(0.24928935982261047),
 np.float64(0.24928935963549276),
 np.float64(0.2492893557683933),
 np.float64(0.24928929915834963),
 np.float64(0.2492886751893025),
 np.float64(0.24928330207958455),
 np.float64(0.2492462852336602),
 np.float64(0.24903902630299055),
 np.float64(0.24808569900431432),
 np.float64(0.24460820014975931),
 np.float64(0.2349439695781523),
 np.float64(0.21475641513175905),
 np.float64(0.1826713919662099),
 np.float64(0.14272522447072042),
 np.float64(0.10227042692681909),
 np.float64(0.06762809950564572),
 np.float64(0.04166472439122743),
 np.float64(0.024151134340423368),
 np.float64(0.013287462679601604)]

In [8]:

# Part 2: Give an integer between 0 and 20 which is the answer to 2.

# Find smallest T such that P(N >= 10 | Y >= T) >= 0.9
# i.e. P(N < 10 | Y >= T) <= 0.1

problem12_T = next(
    T for T, prob in enumerate(problem11_probabilities)
    if prob <= 0.1
)

problem12_T

17

---
## Exam vB, PROBLEM 2
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [9]:

def problem2_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """
    
    """
    Linear Congruential Generator producing integers in {0, 1, ..., M-1}
    """
    M = 2147483647
    a = 1103515245
    b = 12345
    
    out = []
    u = seed
    
    for _ in range(size):
        u = (a * u + b) % M
        out.append(u)
    
    return out


In [10]:

def problem2_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem2_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    
    """
    Produces uniform [0,1] samples using an integer-valued generator
    """
    ints = generator(size=size, seed=seed)
    out = [x / period for x in ints]
    return out


In [11]:

def problem2_accept_reject(uniformGenerator=None, size=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """
    
    """
    Accept-Reject sampler for p0(x) = (pi/2)*|sin(2*pi*x)|
    """
    out = []
    current_seed = seed
    
    while len(out) < size:
        # draw two uniforms
        u = uniformGenerator(size=2, seed=current_seed)
        current_seed += 2
        
        x = u[0]
        y = u[1]
        
        # accept-reject condition
        if y <= abs(np.sin(2 * np.pi * x)):
            out.append(x)
    
    return out

---
#### Local Test for Exam vB, PROBLEM 2
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [13]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem2_LCG(size=10, seed = 1))

period = 2147483647

print("Uniform sampler %s" % problem2_uniform(generator=problem2_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem2_uniform(generator=problem2_LCG, period = period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem2_accept_reject(uniformGenerator = uniform_sampler,size=20,seed=1))

LCG output: [1103527590, 944465040, 1695244727, 1008001095, 235077491, 776026401, 964188548, 19180611, 1753372583, 504528511]
Uniform sampler [0.5138700783782965, 0.4398008065483537, 0.7894098422440746, 0.46938708772388615, 0.1094664871271078, 0.36136545304272577, 0.44898528067813503, 0.008931668013768115, 0.816477734510078, 0.23493939602511907]
Accept-Reject sampler [0.5415987379577005, 0.5970560571165086, 0.6525133762753165, 0.6802420358547205, 0.7079706954341245, 0.7356993550135286, 0.7634280145929325, 0.7911566741723366, 0.8188853337517406, 0.8466139933311445, 0.9020713124899525, 0.0961719295457806, 0.15162924870458863, 0.17935790828399262, 0.20708656786339663, 0.23481522744280064, 0.26254388702220466, 0.29027254660160867, 0.34572986576041664, 0.37345852533982066]


---
## Exam vB, PROBLEM 3
Maximum Points = 8


## Concentration of measure

As you recall, we said that concentration of measure was simply the phenomenon where we expect that the probability of a large deviation of some quantity becoming smaller as we observe more samples: [0.4 points per correct answer]

1. Which of the following will exponentially concentrate, i.e. for some $C_1,C_2,C_3,C_4 $ 
$$
    P(Z - \mathbb{E}[Z] \geq \epsilon) \leq C_1 e^{-C_2 n \epsilon^2} \wedge C_3 e^{-C_4 n (\epsilon+1)} \enspace .
$$

    1. The empirical mean of i.i.d. sub-Gaussian random variables?
    2. The empirical mean of i.i.d. sub-Exponential random variables?
    3. The empirical mean of i.i.d. random variables with finite variance?
    4. The empirical variance of i.i.d. random variables with finite variance?
    5. The empirical variance of i.i.d. sub-Gaussian random variables?
    6. The empirical variance of i.i.d. sub-Exponential random variables?
    7. The empirical third moment of i.i.d. sub-Gaussian random variables?
    8. The empirical fourth moment of i.i.d. sub-Gaussian random variables?
    9. The empirical mean of i.i.d. deterministic random variables?
    10. The empirical tenth moment of i.i.d. Bernoulli random variables?

2. Which of the above will concentrate in the weaker sense, that for some $C_1$
$$
    P(Z - \mathbb{E}[Z] \geq \epsilon) \leq \frac{C_1}{n \epsilon^2}?
$$

## Concentration of Measure — First Set (Strict Interpretation)

### Key Ideas

- **Exponential concentration**: requires strong tail control (sub-Gaussian or sub-Exponential) and independence. Empirical means of sub-Gaussian/sub-Exponential variables and deterministic/bounded quantities can be guaranteed to concentrate exponentially.  
- **Polynomial (Chebyshev-type) concentration**: requires only finite variance (or finite variance of the quantity). Deterministic quantities are trivial.

We do **not assume anything beyond what is explicitly stated**.

---

## Part 1: Exponential Concentration

Check whether 
$$P(Z - \mathbb{E}[Z] \ge \epsilon) \le C_1 e^{-C_2 n \epsilon^2} \vee C_3 e^{-C_4 n (\epsilon+1)}$$
applies under the given assumptions.

### Item-by-item reasoning

1. **Empirical mean of i.i.d. sub-Gaussian random variables**  
   Sub-Gaussian assumption guarantees exponential concentration of the mean (Hoeffding inequality).  
   ✅ Exponentially concentrates

2. **Empirical mean of i.i.d. sub-Exponential random variables**  
   Bernstein-type inequalities apply.  
   ✅ Exponentially concentrates

3. **Empirical mean of i.i.d. random variables with finite variance**  
   Finite variance alone does not give exponential tails.  
   ❌ Does not exponentially concentrate

4. **Empirical variance of i.i.d. sub-Gaussian random variables**  
   Quadratic forms of sub-Gaussian RVs concentrate exponentially.  
   ✅ Exponentially concentrates

5. **Empirical variance of i.i.d. sub-Exponential random variables**  
   Sub-Exponential assumption is not sufficient for variance; exponential concentration not guaranteed.  
   ❌ Does not exponentially concentrate

6. **Empirical variance of i.i.d. random variables with finite variance**  
   Finite variance alone is insufficient for exponential concentration.  
   ❌ Does not exponentially concentrate

7. **Empirical third moment of i.i.d. sub-Gaussian random variables**  
   Exponential concentration of higher moments is not guaranteed.  
   ❌ Does not exponentially concentrate

8. **Empirical fourth moment of i.i.d. sub-Gaussian random variables**  
   Same reasoning as above.  
   ❌ Does not exponentially concentrate

9. **Empirical mean of i.i.d. deterministic random variables**  
   Trivial concentration (no randomness).  
   ✅ Exponentially concentrates

10. **Empirical tenth moment of i.i.d. Bernoulli random variables**  
    Bounded → sub-Gaussian → empirical moments concentrate exponentially.  
    ✅ Exponentially concentrates

### Answer — Exponential Concentration

- 1, 2, 4, 9, 10

---

## Part 2: Polynomial (Weaker) Concentration

Check whether 
$$P(Z - \mathbb{E}[Z] \ge \epsilon) \le \frac{C_1}{n \epsilon^2}$$
applies under the given assumptions.

- Sub-Gaussian or sub-Exponential variables have finite variance → polynomial concentration holds: 1,2,4,5,7,8,10  
- Finite variance → Chebyshev inequality applies: 3,6  
- Deterministic variables trivially satisfy polynomial bound: 9  

### Answer — Polynomial Concentration

- 1,2,3,4,5,6,7,8,9,10

---

## Summary Table

| #  | Quantity                                     | Exponential | Polynomial |
|----|---------------------------------------------|------------|------------|
| 1  | Mean, sub-Gaussian                          | Yes        | Yes        |
| 2  | Mean, sub-Exponential                       | Yes        | Yes        |
| 3  | Mean, finite variance                        | No         | Yes        |
| 4  | Variance, sub-Gaussian                       | Yes        | Yes        |
| 5  | Variance, sub-Exponential                    | No         | Yes        |
| 6  | Variance, finite variance                    | No         | Yes        |
| 7  | 3rd moment, sub-Gaussian                     | No         | Yes        |
| 8  | 4th moment, sub-Gaussian                     | No         | Yes        |
| 9  | Mean, deterministic                          | Yes        | Yes        |
| 10 | 10th moment, Bernoulli                       | Yes        | Yes        |


In [ ]:

# Answers to part 1, which of the alternatives exponentially concentrate, answer as a list
# i.e. [1,4,5] that is example 1, 4, and 5 concentrate
problem3_answer_1 = [1, 2, 4, 9, 10]

In [ ]:

# Answers to part 2, which of the alternatives concentrate in the weaker sense, answer as a list
# i.e. [1,4,5] that is example 1, 4, and 5 concentrate
problem3_answer_2 = [1,2,3,4,5,6,7,8,9,10]

---
## Exam vB, PROBLEM 4
Maximum Points = 8


## SMS spam filtering [8p]

In the following problem we will explore SMS spam texts. The dataset is the `SMS Spam Collection Dataset` and we have provided for you a way to load the data. If you run the appropriate cell below, the result will be in the `spam_no_spam` variable. The result is a `list` of `tuples` with the first position in the tuple being the SMS text and the second being a flag `0 = not spam` and `1 = spam`.

1. [3p] Let $X$ be the random variable that represents each SMS text (an entry in the list), and let $Y$ represent whether text is spam or not i.e. $Y \in \{0,1\}$. Thus $\mathbb{P}(Y = 1)$ is the probability that we get a spam. The goal is to estimate:
$$
    \mathbb{P}(Y = 1 | \text{"free" or "prize" is in } X) \enspace .
$$
That is, the probability that the SMS is spam given that "free" or "prize" occurs in the SMS. 
Hint: it is good to remove the upper/lower case of words so that we can also find "Free" and "Prize"; this can be done with `text.lower()` if `text` a string.
2. [3p] Provide a "90\%" interval of confidence around the true probability. I.e. use the Hoeffding inequality to obtain for your estimate $\hat P$ of the above quantity. Find $l > 0$ such that the following holds:
$$
    \mathbb{P}(\hat P - l \leq \mathbb{E}[\hat P] \leq \hat P + l) \geq 0.9 \enspace .
$$
3. [2p] Repeat the two exercises above for "free" appearing twice in the SMS.

In [ ]:

# Run this cell to get the SMS text data
from exam_extras import load_sms
spam_no_spam = load_sms()

In [ ]:

# fill in the estimate for part 1 here (should be a number between 0 and 1)

# ---------- Part 1: "free" OR "prize" appears ----------
labels_1 = []
for text, y in spam_no_spam:
    t = text.lower()
    if ("free" in t) or ("prize" in t):
        labels_1.append(y)

n1 = len(labels_1)
problem4_hatP = sum(labels_1) / n1

In [ ]:

# fill in the calculated l from part 2 here

import math

# Hoeffding bound with delta = 0.1 (90% confidence)
delta = 0.1
problem4_l = math.sqrt((1 / (2 * n1)) * math.log(2 / delta))

In [ ]:

# fill in the estimate for hatP for the double free question in part 3 here (should be a number between 0 and 1)

# ---------- Part 3: "free" appears at least twice ----------
labels_2 = []
for text, y in spam_no_spam:
    t = text.lower()
    if t.count("free") >= 2:
        labels_2.append(y)

n2 = len(labels_2)
problem4_hatP2 = sum(labels_2) / n2

In [ ]:

# fill in the estimate for l for the double free question in part 3 here

problem4_l2 = math.sqrt((1 / (2 * n2)) * math.log(2 / delta))

### What this does (brief explanation)

- **Estimator**

  
  $$\hat P = \frac{\text{\# spam messages satisfying condition}}{\text{\# messages satisfying condition}}$$
  

- **Hoeffding bound (Bernoulli variables)**  
  For confidence level $1 - \delta = 0.9$:

  
  $$l = \sqrt{\frac{1}{2n}\ln\left(\frac{2}{\delta}\right)}$$
 

- **Uses:**
  - Case-insensitive matching (`text.lower()`)
  - Substring counting for `"free"` appearing twice


---
## Exam vB, PROBLEM 5
Maximum Points = 8


## Markovian travel

The dataset `Travel Dataset - Datathon 2019` is a simulated dataset designed to mimic real corporate travel systems -- focusing on flights and hotels. The file is at `data/flights.csv` in the same folder as `Exam.ipynb`, i.e. you can use the path `data/flights.csv` from the notebook to access the file.

1. [2p] In the first code-box 
    1. Load the csv from file `data/flights.csv`
    2. Fill in the value of the variables as specified by their names.
2. [2p] In the second code-box your goal is to estimate a Markov chain transition matrix for the travels of these users. For example, if we enumerate the cities according to alphabetical order, the first city `'Aracaju (SE)'` would correspond to $0$. Each row of the file corresponds to one flight, i.e. it has a starting city and an ending city. We model this as a stationary Markov chain, i.e. each user's travel trajectory is a realization of the Markov chain, $X_t$. Here, $X_t$ is the current city the user is at, at step $t$, and $X_{t+1}$ is the city the user travels to at the next time step. This means that to each row in the file there is a corresponding pair $(X_{t},X_{t+1})$. The stationarity assumption gives that for all $t$ there is a transition density $p$ such that $P(X_{t+1} = y | X_t = x) = p(x,y)$ (for all $x,y$). The transition matrix should be `n_cities` x `n_citites` in size.
3. [2p] Use the transition matrix to compute out the stationary distribution.
4. [2p] Given that we start in 'Aracaju (SE)' what is the probability that after 3 steps we will be back in 'Aracaju (SE)'?

In [2]:

import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("data/flights.csv")

# Basic quantities
number_of_cities = df["from"].nunique()
number_of_userCodes = df["userCode"].nunique()
number_of_observations = len(df)

In [3]:

# This is a very useful function that you can use for part 2. You have seen this before when parsing the
# pride and prejudice book.

def makeFreqDict(myDataList):
    '''Make a frequency mapping out of a list of data.

    Param myDataList, a list of data.
    Return a dictionary mapping each unique data value to its frequency count.'''

    freqDict = {} # start with an empty dictionary

    for res in myDataList:
        if res in freqDict: # the data value already exists as a key
                freqDict[res] = freqDict[res] + 1 # add 1 to the count using sage integers
        else: # the data value does not exist as a key value
            freqDict[res] = 1 # add a new key-value pair for this new data value, frequency 1

    return freqDict # return the dictionary created

In [4]:

# Collect all cities appearing as origins
cities = list(df["from"])

unique_cities = sorted(set(cities)) # The unique cities
n_cities = len(unique_cities) # The number of unique citites

# Count the different transitions

# Build transition list (X_t, X_{t+1})
transitions = list(zip(df["from"], df["to"])) # A list containing tuples ex: ('Aracaju (SE)','Rio de Janeiro (RJ)') of all transitions in the text

# Count transitions
transition_counts = makeFreqDict(transitions) # A dictionary that counts the number of each transition 

# ex: ('Aracaju (SE)','Rio de Janeiro (RJ)'):4
indexToCity = {i: city for i, city in enumerate(unique_cities)} # A dictionary that maps the n-1 number to the n:th unique_city,
# ex: 0:'Aracaju (SE)'
cityToIndex = {city: i for i, city in indexToCity.items()} # The inverse function of indexToWord, 
# ex: 'Aracaju (SE)':0

# Part 3, finding the maximum likelihood estimate of the transition matrix

# Initialize transition matrix
transition_matrix = np.zeros((n_cities, n_cities)) # a numpy array of size (n_cities,n_cities)

# Count outgoing transitions per city
outgoing_counts = makeFreqDict(df["from"])

# The transition matrix should be ordered in such a way that
# p_{'Aracaju (SE)','Rio de Janeiro (RJ)'} = transition_matrix[cityToIndex['Aracaju (SE)'],cityToIndex['Rio de Janeiro (RJ)']]
# and represents the probability of travelling Aracaju (SE)->Rio de Janeiro (RJ)

# Fill transition matrix with MLE probabilities
for (city_from, city_to), count in transition_counts.items():
    i = cityToIndex[city_from]
    j = cityToIndex[city_to]
    transition_matrix[i, j] = count / outgoing_counts[city_from]

# Make sure that the transition_matrix does not contain np.nan from division by zero for instance

# Ensure no NaNs
transition_matrix = np.nan_to_num(transition_matrix)

In [5]:

# This should be a numpy array of length n_cities which sums to 1 and is all positive

# Eigenvector method
eigvals, eigvecs = np.linalg.eig(transition_matrix.T)

# Find eigenvector corresponding to eigenvalue 1
idx = np.argmin(np.abs(eigvals - 1))
stationary = np.real(eigvecs[:, idx])

# Normalize
stationary_distribution_problem5 = stationary / stationary.sum()

In [6]:

# Compute the return probability for part 3 of problem 5

P3 = np.linalg.matrix_power(transition_matrix, 3)

aracaju_index = cityToIndex['Aracaju (SE)']

return_probability_problem5 = P3[aracaju_index, aracaju_index]

---
#### Local Test for Exam vB, PROBLEM 5
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [7]:
# Once you have created all your functions, you can make a small test here to see
# what would be generated from your model.
import numpy as np

start = np.zeros(shape=(n_cities,1))
start[cityToIndex['Aracaju (SE)'],0] = 1

current_pos = start
for i in range(10):
    random_word_index = np.random.choice(range(n_cities),p=current_pos.reshape(-1))
    current_pos = np.zeros_like(start)
    current_pos[random_word_index] = 1
    print(indexToCity[random_word_index],end='->')
    current_pos = (current_pos.T@transition_matrix).T

Aracaju (SE)->Recife (PE)->Campo Grande (MS)->Sao Paulo (SP)->Salvador (BH)->Natal (RN)->Recife (PE)->Florianopolis (SC)->Sao Paulo (SP)->Natal (RN)->

---
## Exam vB, PROBLEM 6
Maximum Points = 8


## Black box testing

In the following problem we will continue with our SMS spam / nospam data. This time we will try to approach the problem as a pattern recognition problem. For this particular problem I have provided you with everything -- data is prepared, split into train-test sets and a black-box model has been fitted on the training data and predicted on the test data. Your goal is to calculate test metrics and provide guarantees for each metric.

1. [2p] Compute precision for class 1 (see notes 8.3.2 for definition), then provide an interval using Hoeffding's inequality for a 95\% confidence.
2. [2p] Compute recall for class 1(see notes 8.3.2 for definition), then provide an interval using Hoeffding's inequality for a 95\% interval.
3. [2p] Compute accuracy (0-1 loss), then provide an interval using Hoeffding's inequality for a 95\% interval.
4. [2p] If we would have used a classifier with VC-dimension 3, would we have obtained a smaller interval for accuracy by using all data?

In [ ]:

# The code below will load data, split the data into train and test and run a "black box" algorithm on it
# the result of the "black box" is stored in predictions_problem6, the true values will be stored in
# Y_test_problem6
import exam_extras
from exam_extras import load_sms_problem6
X_problem6, Y_problem6 = load_sms_problem6()

X_train_problem6,X_test_problem6,Y_train_problem6,Y_test_problem6 = exam_extras.train_test_split(X_problem6,Y_problem6)
predictions_problem6 = exam_extras.knn_predictions(X_train_problem6,Y_train_problem6,X_test_problem6,k=4)

In [ ]:
import numpy as np
import math

# Compute the precision of predictions_problem6 with respect to Y_test_problem6

pred = np.array(predictions_problem6)
true = np.array(Y_test_problem6)

# Precision for class 1
tp = np.sum((pred == 1) & (true == 1))
fp = np.sum((pred == 1) & (true == 0))

problem6_precision = tp / (tp + fp)

In [ ]:

# Compute the interval length l of precision of predictions_problem6 with respect to Y_test_problem6, with the same definition of l as in problem 4

# Hoeffding interval (95%)
delta = 0.05
n_precision = tp + fp
problem6_precision_l = math.sqrt((1 / (2 * n_precision)) * math.log(2 / delta))

In [ ]:

# Repeat the same procedure but for recall

fn = np.sum((pred == 0) & (true == 1))

problem6_recall = tp / (tp + fn)

In [ ]:

n_recall = tp + fn
problem6_recall_l = math.sqrt((1 / (2 * n_recall)) * math.log(2 / delta))

In [ ]:

# Repeat the same procedure but for accuracy or 0-1 loss

correct = np.sum(pred == true)
n_test = len(true)

problem6_accuracy = correct / n_test

In [ ]:

problem6_accuracy_l = math.sqrt((1 / (2 * n_test)) * math.log(2 / delta))

In [ ]:

# Below you will calculate the interval parameter l for a classifier running on all data with a VC dimension of 3
# put the value in problem6_VC_l and answer problem_VC_smaller as True if the interval is smaller than the test-accuracy above
# if not answer False. Make sure you replace XXX with something even if you only answer one of them.
# VC-dimension bound

d = 3
n_all = len(Y_problem6)

problem6_VC_l = math.sqrt(
    (8 / n_all) * (math.log(4 / delta) + d * math.log((2 * math.e * n_all) / d))
)

# Is the VC interval smaller than the test Hoeffding interval?
problem6_VC_smaller = problem6_VC_l < problem6_accuracy_l

## Part 1 — Precision (class 1) with 95% Hoeffding interval

### Definition

Precision for class 1 is defined as

$$
\text{Precision} = P(Y = 1 \mid \hat Y = 1)
$$

That is, among all samples predicted as spam, how many are actually spam.

---

## Part 2 — Recall (class 1) with 95% Hoeffding interval

### Definition

Recall for class 1 is defined as

$$
\text{Recall} = P(\hat Y = 1 \mid Y = 1)
$$

That is, among all true spam messages, how many are correctly identified as spam by the classifier.

---

## Part 3 — Accuracy (0–1 loss) with 95% Hoeffding interval

### Definition

Accuracy (also called 0–1 accuracy) is defined as the fraction of correct predictions:

$$
\text{Accuracy} = \frac{1}{n} \sum_{i=1}^{n} \mathbf{1}\{\hat Y_i = Y_i\}
$$

This measures the overall probability that the classifier predicts the correct label.

---

## Confidence intervals via Hoeffding’s inequality

For a bounded random variable $X \in [0,1]$ and $n$ independent samples, Hoeffding’s inequality gives

$$
P\left( |\hat{P} - \mathbb{E}[\hat{P}]| \le l \right) \ge 1 - \delta
$$

where

$$
l = \sqrt{\frac{1}{2n} \ln\left(\frac{2}{\delta}\right)}
$$

This bound is distribution-free and holds for precision, recall, and accuracy since they are averages of Bernoulli random variables.

---

## Part 4 — VC-dimension bound comparison

### VC generalization bound

For a hypothesis class with VC-dimension $d$, a standard uniform convergence bound for accuracy is

$$
l_{\text{VC}} =
\sqrt{
\frac{8}{n}
\left(
\log\frac{4}{\delta}
+ d \log\frac{2 e n}{d}
\right)
}
$$

This bound holds uniformly over the entire hypothesis class and does not depend on the observed labels.

---

## Interpretation

- VC-dimension bounds are **distribution-free**, making them more conservative.
- Hoeffding bounds computed on a test set are typically **tighter**.
- Even when using all data, a low-VC classifier does **not necessarily** yield a smaller confidence interval than a test-based Hoeffding bound.

---

## Final conclusion

Using a classifier with VC-dimension 3 on all available data would **not generally produce a smaller confidence interval** for accuracy compared to the Hoeffding interval computed on the test set.
